#### Measuring load and inference times for GoogLeNet models

In [1]:
!pip install pickle5

In [2]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, regularizers, callbacks

import os
import time
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [3]:
tf.__version__

'2.3.4'

In [4]:
tf.config.list_physical_devices(device_type=None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# specify variables
path = 'processed_data/WM-clean60'
result_path = 'results'

filename = 'WM-clean60'
map_column = 'waferMap60'

model_id = 'yutandem'
data_id = '60'
note = '' # -optional

In [6]:
# load test set(s) to be used
# directly from S3 (using boto3 resource)
start = time.time()

test_key = f'{path}/{filename}-test.pkl'
test = pickle.loads(my_bucket.Object(test_key).get()['Body'].read())

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f"Test: {len(test)}")

Wall time: 1.93 seconds
Test: 25943


##### Data set-up

In [7]:
# prepare inputs
start = time.time()

x_det = np.stack(test[map_column])
x_det = tf.expand_dims(x_det, axis=3, name=None)
x_det = tf.repeat(x_det, 3, axis=3)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_det.shape

Wall time: 1.67 seconds


TensorShape([25943, 60, 60, 3])

In [8]:
# prepare labels for evaluating results
start = time.time()

y_det = np.asarray(test['detectLabels']).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

Wall time: 0.00 seconds


#### Load and run GoogLeNet models

###### 60x60 no filter

In [15]:
# load saved detect model from S3
start = time.time()

all60 = s3_get_keras_model('googlenet-all60')
all60.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
resizing (Resizing)             (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        resizing[0][0]                   
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 55, 55, 64)   0           conv2d[0][0]                     
_______________________________________________________________________________________

In [16]:
# generate predictions
start = time.time()

all60_pred = all60.predict(x_det)
all60_labels = np.argmax(all60_pred[0], axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

Wall time: 18.65 seconds


###### 60x60 with 3x3 median filter

In [11]:
# load saved detect model from S3
start = time.time()

all60mfilter3 = s3_get_keras_model('googlenet-all60-mfilter3')
all60mfilter3.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
resizing (Resizing)             (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        resizing[0][0]                   
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 55, 55, 64)   0           conv2d[0][0]                     
_______________________________________________________________________________________

In [12]:
# generate predictions
start = time.time()

all60mfilter3_pred = all60mfilter3.predict(x_det)
all60mfilter3_labels = np.argmax(all60mfilter3_pred[0], axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

Wall time: 18.66 seconds


###### 60x60 with n=2 morphological thinning

In [13]:
# load saved detect model from S3
start = time.time()

all60thin2 = s3_get_keras_model('googlenet-all60-thin2')
all60thin2.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
resizing (Resizing)             (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        resizing[0][0]                   
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 55, 55, 64)   0           conv2d[0][0]                     
_______________________________________________________________________________________

In [14]:
# generate predictions
start = time.time()

all60thin2_pred = all60thin2.predict(x_det)
all60thin2_labels = np.argmax(all60thin2_pred[0], axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

Wall time: 18.61 seconds
